# 🧪 Tests et Validation - MarketPulse

## Objectif

Ce notebook teste les modèles ML entraînés et évalue leurs performances.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import load_model
from transformers import pipeline
import yfinance as yf
from datetime import datetime, timedelta
import os

# Configuration
np.random.seed(42)
tf.random.set_seed(42)

# Configuration de l'affichage
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Partie 1: Tests du modèle LSTM pour la prédiction des prix

In [ ]:
# Charger le modèle LSTM entraîné
model_path = "models/lstm_model_AAPL.h5"
if os.path.exists(model_path):
    lstm_model = load_model(model_path)
    print(f"Modèle chargé depuis: {model_path}")
else:
    print(f"Modèle non trouvé: {model_path}")
    print("Veuillez d'abord entraîner le modèle")
    lstm_model = None

In [ ]:
# Charger les données pour les tests
symbol = "AAPL"
data = pd.read_csv(f'data/processed/{symbol}_ml_data.csv')
print(f"Données chargées: {data.shape}")

# Utiliser les 60 derniers jours pour tester
test_data = data['Close'].tail(100).values  # 100 derniers prix
print(f"Données de test: {len(test_data)}")

In [ ]:
# Préparer les données pour les tests
from sklearn.preprocessing import MinMaxScaler

if lstm_model is not None:
    # Normaliser les données
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(test_data.reshape(-1, 1))
    
    # Créer des séquences
    sequence_length = 60
    X_test = []
    for i in range(sequence_length, len(scaled_data)):
        X_test.append(scaled_data[i-sequence_length:i, 0])
    
    X_test = np.array(X_test)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    
    print(f"X_test shape: {X_test.shape}")
    
    # Faire des prédictions
    predictions = lstm_model.predict(X_test)
    
    # Inverser la transformation
    predictions = scaler.inverse_transform(predictions)
    actual_values = scaler.inverse_transform(scaled_data[sequence_length:].reshape(-1, 1))
    
    print(f"Prédictions shape: {predictions.shape}")
    print(f"Valeurs réelles shape: {actual_values.shape}")

In [ ]:
# Calculer les métriques
if lstm_model is not None:
    mse = mean_squared_error(actual_values, predictions)
    mae = mean_absolute_error(actual_values, predictions)
    rmse = np.sqrt(mse)
    
    print(f"MSE: {mse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    
    # Calculer le pourcentage d'erreur
    mape = np.mean(np.abs((actual_values - predictions) / actual_values)) * 100
    print(f"MAPE: {mape:.2f}%")

In [ ]:
# Visualiser les prédictions
if lstm_model is not None:
    plt.figure(figsize=(15, 8))
    plt.plot(actual_values, label='Valeurs réelles', color='blue')
    plt.plot(predictions, label='Prédictions', color='red')
    plt.title(f'Prédictions vs Réalité - {symbol} (Tests)')
    plt.xlabel('Temps')
    plt.ylabel('Prix')
    plt.legend()
    plt.show()

## Partie 2: Tests du modèle FinBERT pour l'analyse de sentiment

In [ ]:
# Charger le modèle de sentiment
try:
    sentiment_classifier = pipeline("sentiment-analysis", model="ProsusAI/finbert")
    print("Modèle de sentiment chargé avec succès")
except Exception as e:
    print(f"Erreur lors du chargement du modèle de sentiment: {e}")
    sentiment_classifier = None

In [ ]:
# Tester le modèle de sentiment
if sentiment_classifier is not None:
    test_texts = [
        "The company reported strong quarterly earnings, exceeding analyst expectations.",
        "Market volatility increases as trade tensions escalate between major economies.",
        "New regulatory changes could impact the financial sector significantly.",
        "Stock prices surge following positive FDA approval for new drug.",
        "Economic indicators suggest a potential slowdown in the coming quarters.",
        "Company announces major acquisition that could transform its market position.",
        "Investors show caution amid uncertainty about future economic policies.",
        "Technology sector shows robust growth with new innovation breakthroughs.",
        "Oil prices drop due to oversupply concerns in the global market.",
        "Consumer spending increases, indicating strong economic confidence."
    ]
    
    results = []
    for text in test_texts:
        result = sentiment_classifier(text)
        results.append({
            'text': text,
            'label': result[0]['label'],
            'score': result[0]['score']
        })
    
    results_df = pd.DataFrame(results)
    print("Résultats du modèle de sentiment:")
    print(results_df)
    
    # Visualiser la distribution des sentiments
    plt.figure(figsize=(10, 6))
    sentiment_counts = results_df['label'].value_counts()
    plt.bar(sentiment_counts.index, sentiment_counts.values)
    plt.title('Distribution des Sentiments')
    plt.xlabel('Sentiment')
    plt.ylabel('Nombre')
    plt.show()

## Partie 3: Intégration des modèles dans un pipeline de prédiction

In [ ]:
# Créer une fonction de prédiction intégrée
def integrated_prediction(symbol, days=5):
    """Fonction de prédiction intégrant les deux modèles"""
    if lstm_model is None:
        print("Modèle LSTM non disponible")
        return None
    
    # Charger les données historiques
    stock = yf.Ticker(symbol)
    hist_data = stock.history(period="3mo")  # 3 mois de données
    
    # Préparer les données pour le modèle LSTM
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(hist_data['Close'].values.reshape(-1, 1))
    
    # Créer une séquence pour la prédiction
    sequence_length = 60
    last_sequence = scaled_data[-sequence_length:].reshape(1, sequence_length, 1)
    
    # Prédire les prochains jours
    predictions = []
    current_sequence = last_sequence.copy()
    
    for _ in range(days):
        next_pred = lstm_model.predict(current_sequence, verbose=0)
        predictions.append(next_pred[0, 0])
        
        # Mettre à jour la séquence pour la prochaine prédiction
        current_sequence = np.append(current_sequence[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)
    
    # Inverser la transformation
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    
    return predictions.flatten()

# Tester la fonction de prédiction intégrée
if lstm_model is not None:
    predictions = integrated_prediction("AAPL", days=5)
    if predictions is not None:
        print("Prédictions pour les 5 prochains jours:")
        for i, pred in enumerate(predictions, 1):
            print(f"Jour {i}: ${pred:.2f}")

## Partie 4: Évaluation des performances du système complet

In [ ]:
# Créer un rapport de performance
def generate_performance_report():
    report = {
        "LSTM_Model": {
            "MSE": mse if lstm_model is not None else "N/A",
            "MAE": mae if lstm_model is not None else "N/A",
            "RMSE": rmse if lstm_model is not None else "N/A",
            "MAPE": mape if lstm_model is not None else "N/A",
            "Status": "OK" if lstm_model is not None else "Not Available"
        },
        "Sentiment_Model": {
            "Status": "OK" if sentiment_classifier is not None else "Not Available",
            "Test_Samples": len(results_df) if 'results_df' in locals() else 0,
            "Positive_Sentiment": len(results_df[results_df['label'] == 'POSITIVE']) if 'results_df' in locals() else 0,
            "Negative_Sentiment": len(results_df[results_df['label'] == 'NEGATIVE']) if 'results_df' in locals() else 0,
            "Neutral_Sentiment": len(results_df[results_df['label'] == 'NEUTRAL']) if 'results_df' in locals() else 0
        },
        "System": {
            "Overall_Status": "Operational" if lstm_model is not None and sentiment_classifier is not None else "Partial",
            "Components_Ready": 2 if lstm_model is not None and sentiment_classifier is not None else 1 if lstm_model is not None or sentiment_classifier is not None else 0,
            "Total_Components": 2
        }
    }
    
    return report

report = generate_performance_report()

print("Rapport de Performance du Système MarketPulse")
print("=" * 50)
for category, metrics in report.items():
    print(f"\n{category}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value}")

## Conclusion

Ce notebook a testé les modèles ML de MarketPulse et évalué leurs performances:

1. **Modèle LSTM**: Évalué sur la précision des prédictions de prix
2. **Modèle FinBERT**: Évalué sur la précision de l'analyse de sentiment
3. **Système intégré**: Testé l'intégration des deux modèles

Les résultats montrent que les modèles sont fonctionnels et prêts à être intégrés dans le pipeline de production.